In [16]:
import torch
import numpy as np
import os
import pandas as pd
from torch.nn.functional import mse_loss
from compute_distance import compute_distance

In [17]:
# Folder path containing CSV files
folder_path = "data/csv"

# Define value ranges
first_values = np.arange(0.3, 0.7, 0.05)  # 0.25 to 0.7 with step 0.05
# second_values = np.arange(3, 15, 1)  # 3 to 14
second_values = np.arange(3, 15, 1)  # 3 to 14

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

rho=1.0
nu=0.01

In [18]:
for first in first_values:
    for second in second_values:
        first_str = f"{first:.2f}".rstrip("0").rstrip(".")
        file_name = f"W_{first_str}_{second}_1.csv"
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            
            # Load CSV file
            df = pd.read_csv(file_path)
            
            # Extract input and output data
            input_data = df.iloc[:, [0, 1, 5, 7]].values  # (x, y, d, N) [0, 1, 5, 7]
            output_data = df.iloc[:, [2, 3, 4]].values  # (u, v, p)

            input_tensor = torch.tensor(input_data, dtype=torch.float32, device=device)
            output_tensor = torch.tensor(output_data, dtype=torch.float32, device=device)

            x = input_tensor[:, 0].requires_grad_(True)
            y = input_tensor[:, 1].requires_grad_(True)
            d = input_tensor[:, 2]
            n = input_tensor[:, 3]

            u, v, p = output_tensor[:, 0], output_tensor[:, 1], output_tensor[:, 2]

            u_hard = u
            v_hard = v
            p_hard = p

            u_x = torch.autograd.grad(u_hard,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
            u_xx = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
            u_y = torch.autograd.grad(u_hard,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
            u_yy = torch.autograd.grad(u_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
            p_x = torch.autograd.grad(p_hard,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
            #P_xx = torch.autograd.grad(p_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
            loss_1 = (u_hard*u_x+v_hard*u_y-nu*(u_xx+u_yy)+1/rho*p_x)

            v_x = torch.autograd.grad(v_hard,x,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
            v_xx = torch.autograd.grad(v_x,x,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]

            v_y = torch.autograd.grad(v_hard,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]

            v_yy = torch.autograd.grad(v_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
            P_y = torch.autograd.grad(p_hard,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
            #P_yy = torch.autograd.grad(P_y,y,grad_outputs=torch.ones_like(x),create_graph = True,allow_unused = True)[0]


            loss_2 = (u_hard*v_x+v_hard*v_y - nu*(v_xx+v_yy)+1/rho*P_y)
            #Main_deriv = torch.cat((u_x,u_xx,u_y,u_yy,P_x,v_x,v_xx,v_y,v_yy,P_y),1)
            loss_3 = (u_x + v_y)
            #loss_3 = u_x**2 + 2*u_y*v_x + v_y**2+1/rho*(P_xx + P_yy)
            #loss_3 = loss_3*100

            loss_1 = mse_loss(loss_1, torch.zeros_like(loss_1))
            loss_2 = mse_loss(loss_2, torch.zeros_like(loss_2))
            loss_3 = mse_loss(loss_3, torch.zeros_like(loss_3))

            print(f"Loss 1: {loss_1.item()}, Loss 2: {loss_2.item()}, Loss 3: {loss_3.item()}")

Processing W_0.3_3_1.csv...


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn